## 1.0 Change working Directory to Root Directory

In [1]:
# Set the repository root as a working directory 
%cd ..

/mnt/c/Users/MR-BEST/course-ds-base-root/course-ds-base


/home/mr-best/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### 0.1 Init DVC repository
 2.1 Init DVC repository and setup DVC remote storage

dvc init

2.2 Add DVC repository under git control

git add .
git commit -m "Init DVC repo"

# 0.2 View config

In [2]:
# Look on stages config 
!cat params.yaml

#### Initial state before the update

# base:
#   random_state: 42
#   log_level: INFO

# data:
  # dataset_csv: 'data/raw/iris.csv'
#   features_path: 'data/processed/featured_iris.csv'
#   test_size: 0.2
#   trainset_path: 'data/processed/train_iris.csv'
#   testset_path: 'data/processed/test_iris.csv'


# train:
#   clf_params:
#     'C': 0.001
#     'solver': 'lbfgs'
#     'multi_class': 'multinomial'
#     'max_iter': 100
#   model_path: 'models/model.joblib'

# reports:
#   metrics_file: 'reports/metrics.json'
#   confusion_matrix_image: 'reports/confusion_matrix.png'

  #### Newer Versions of Metrics

  
base:
  random_state: 42
  log_level: INFO


data_load:
  dataset_csv: 'data/raw/iris.csv'


featurize:
  features_path: 'data/processed/featured_iris.csv'
  target_column: target


data_split:
  test_size: 0.2
  trainset_path: 'data/processed/train_iris.csv'
  testset_path: 'data/processed/test_iris.csv'


train:

  cv: 3
  estimator_name: logreg
  estimators:
    logreg: # skl

###  Create and run stages for a DVC pipeline

##  First Stage of ML Pipeline : Extract and Raw Load Data

dvc stage add -n data_load \
    -d src/stages/data_load.py \
    -o data/raw/iris.csv \
    -p base,data_load \
    python src/stages/data_load.py --config=params.yaml

# Featurization

dvc stage add -n featurize \
    -d src/stages/featurize.py \
    -d data/raw/iris.csv \
    -o data/processed/featured_iris.csv \
    -p base,featurize \
    python src/stages/featurize.py --config=params.yaml

## 3 Split dataset into train/test edited directly into the dvc.yaml config file

data_split:
    cmd: python src/stages/data_split.py --config=params.yaml
    deps:
    - data/processed/featured_iris.csv
    - src/stages/data_split.py
    params:
    - base
    - data_split
    - featurize
    outs:
    - data/processed/test_iris.csv
    - data/processed/train_iris.csv

## 4 Train Model

train:
    cmd: python src/stages/train.py --config=params.yaml
    deps:
    - data/processed/test_iris.csv
    - data/processed/train_iris.csv
    - src/stages/train.py
    params:
    - base
    - train
    outs:
    - models/model.joblib

## 5 Evaluate Model with Test Data

In [22]:
evaluate:
    cmd: python src/stages/evaluate.py --config=params.yaml
    deps:
    - models/model.joblib
    - data/processed/test_iris.csv

    params:
    - base
    - train
    - data_split
    - featurize
    - evaluate
    outs:
    - metrics.json
    - confusion_matrix.png

2024-01-15 10:22:31,528 — TRAIN — INFO — Get model/estimator name
2024-01-15 10:22:31,529 — TRAIN — INFO — The name of Model/Estimator: logreg
2024-01-15 10:22:31,530 — TRAIN — INFO — Load train dataset
2024-01-15 10:22:31,544 — TRAIN — INFO — Train model/estimator
Fitting 3 folds for each of 1 candidates, totalling 3 fits
2024-01-15 10:22:31,577 — TRAIN — INFO — Best score: 0.857564307288572
2024-01-15 10:22:31,578 — TRAIN — INFO — Trained Model Saved


In [ ]:
#  MAnual entry on bash shell

dvc stage add -n evaluate \
    -d models/model.joblib \
    -d data/processed/test_iris.csv\
    -o metrics.json \
    -o confusion_matrix.png\
    -p base, evaluate \
    python src/stages/evaluate.py --config=params.yaml